<a href="https://colab.research.google.com/github/AslanDevbrat/NLP/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Processing

In [1]:
!git clone https://github.com/Neuralearn/deep-learning-with-tensorflow-2

fatal: destination path 'deep-learning-with-tensorflow-2' already exists and is not an empty directory.


In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2022-07-17 11:34:04--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  19.3MB/s    in 8.5s    

2022-07-17 11:34:13 (9.43 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [3]:
!tar -xzf /content/aclImdb_v1.tar.gz

In [48]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.layers import SimpleRNN,LSTM, Dense, Reshape
import numpy as np
import re
import string
import nltk
import numpy as np
from matplotlib import pyplot as plt
import pandas
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
train_directory = "/content/aclImdb/train"
val_directory = "/content/aclImdb/test"

In [4]:
!mv /content/aclImdb/train/unsup /content

mv: cannot stat '/content/aclImdb/train/unsup': No such file or directory


In [5]:
train_dataset = text_dataset_from_directory(
    train_directory,
)

Found 25000 files belonging to 2 classes.


In [6]:
val_dataset = text_dataset_from_directory(
    val_directory,
    shuffle = True
)

Found 25000 files belonging to 2 classes.


In [ ]:
for x, y in train_dataset.take(1):
  print(x)
  print()
  print(y)

In [9]:
def preprocess_sentences(input_data):
    '''
    Input: raw reviews
    output: standardized reviews
    '''
    output=tf.strings.lower(input_data)
    outputs=tf.strings.regex_replace(output,"<[^>]+>","")
    outputs=tf.strings.regex_replace(output,"<[%s]"%re.escape(string.punctuation)," ")
    outputs=tf.strings.regex_replace(output,"  "," ")
    
    return output

In [32]:
def preprocess_sentences(input_data):

     output = tf.strings.lower(input_data)
     output = tf.strings.regex_replace(output, "<[^>]+>","")
     output = tf.strings.regex_replace(output,'[%s]' %re.escape(string.punctuation), "  ")
     output = tf.strings.regex_replace(output, '  '," "  )
     lemmatizer = nltk.stem.WordNetLemmatizer()
     output = tf.strings.join([lemmatizer.lemmatize(str(word)[12:-26], pos = "a") for word in tf.strings.split(output)], separator = " ")
     return output
preprocess_sentences("I kind of consider myself as the #1 fan of Hidden Frontier")


<tf.Tensor: shape=(), dtype=string, numpy=b'i kind of consider myself as the 1 fan of hidden frontier'>

In [11]:
VOCAB_SIZE=2000
SEQUENCE_LENGTH=100
vectorize_layer=TextVectorization(
    standardize=preprocess_sentences,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
)
training_data=train_dataset.map(lambda x,y:x)### inputsxandyand outputsx
vectorize_layer.adapt(training_data)### Adapt the vectorize Layer to the training data

In [12]:
def vectorizer(review, label):
  return tf.one_hot(vectorize_layer(review), depth = VOCAB_SIZE), label


In [13]:
train_dataset = train_dataset.map(vectorizer)
val_dataset = val_dataset.map(vectorizer)

In [17]:
for x , y in train_dataset.take(1):
  print(y)

tf.Tensor([0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0], shape=(32,), dtype=int32)


## Modeling


In [31]:
class RNNCell(tf.keras.layers.Layer):
  def __init__(self, units = 4, hidden_size = 7,):
    super(RNNCell, self).__init__()

    self.units = units
    self.hidden_size = hidden_size

  def build(self, input_shape):
    print(input_shape)

    self.w_hh = self.add_weight(
        shape = (self.hidden_size, self.hidden_size),
        initializer = "random_normal",
        trainable = True,
    )
    self.w_hx = self.add_weight(
        shape = (input_shape[-1], self.hidden_size),
        initializer = "random_normal",
        trainable = True,
    )
    self.w_yh = self.add_weight(
        shape = (self.hidden_size, self.units),
        initializer = "random_normal",
        trainable = True,
    )

    self.b_h = self.add_weight(
        shape = (self.hidden_size,),
        initializer = "random_normal",
        trainable = True,
    )
    
    self.b_y = self.add_weight(
        shape = (self.units,),
        initializer = "random_normal",
        trainable = True,
    )

  def call(self, inputs, h_prev = None):
    if(h_prev == None):
      h_prev = tf.zeros([inputs.shape[0], self.hidden_size])
    h = tf.nn.tanh(tf.matmul(h_prev, self.w_hh) + tf.matmul(inputs, self.w_hx) + self.b_h)
    y = tf.nn.tanh(tf.matmul(h, self.w_yh) + self.b_y)
    
    return tf.constant(h), tf.constant(y)


In [32]:
input = tf.zeros([8,  2000])
layer = RNNCell()
h,y = layer(input)
h.shape, y.shape

(8, 2000)


(TensorShape([8, 7]), TensorShape([8, 4]))

In [41]:
class RNN(tf.keras.layers.Layer):
  def __init__(self, units, ):
    super(RNN, self).__init__()
    self.rnn = RNNCell(units)

  def call(self, inputs):
    output = []

    h,y = self.rnn(input[:,0,:])
    output.append(y)

    for i in range(1, input.shape[-2]):
      h, y = self.rnn(input[:,i,:], h)
      output.append(y)
    shape = np.array(output).shape
    return tf.reshape(output,[ shape[1], shape[0], shape[2]])


In [42]:
input = tf.zeros([8, 100,2000])
layer = RNN(4)
output = layer(input)
output.shape

(8, 2000)


TensorShape([8, 100, 4])

In [46]:
input = tf.zeros([8, 100,2000])
layer = SimpleRNN(4, activation = 'tanh', use_bias = True, return_sequences=False, name = "layer1")

output = layer(input)
output.shape

TensorShape([8, 4])

In [55]:
 input = tf.keras.layers.Input(shape = (SEQUENCE_LENGTH, VOCAB_SIZE))

 model = tf.keras.models.Sequential([
                                     input,
                                     SimpleRNN(4, activation = 'tanh', use_bias = True, return_sequences=True, name = "layer1"),
                                     Dense(1, activation = "relu", name="layer2"),
                                     Reshape((SEQUENCE_LENGTH,), name = "layer3"),
                                     Dense(1, activation = "sigmoid", name = "layer4")
 ])
 model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (SimpleRNN)          (None, 100, 4)            8020      
                                                                 
 layer2 (Dense)              (None, 100, 1)            5         
                                                                 
 layer3 (Reshape)            (None, 100)               0         
                                                                 
 layer4 (Dense)              (None, 1)                 101       
                                                                 
Total params: 8,126
Trainable params: 8,126
Non-trainable params: 0
_________________________________________________________________
